In [2]:
import pandas as pd
import glob as glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('white')

### Set parameters

In [7]:
N_samples = 10000
threshold = .5 
REWARD = 1

### Free parameters to fit

In [67]:
COST = 0.5
B=6
A=60 
u_skill = .625 #mean of beta
tao_skill = 20 #variance of beta
skill=.4
scale = 200

### Functions

In [39]:
def prob_reward(e,diff,skill, A, B):
    return 1/(1 + A * np.exp(-(e * skill / diff) * B)) 

def get_effort(reward, cost, diff, skill, A, B):
    Us=list()
    es = np.linspace(0, 1, 100) 
    for i in es:
        U=reward*prob_reward(i, diff, skill, A, B)-cost*i #utility function
        Us.append(U)
    return es[np.argmax(Us)], Us, es  #return argmax of utility, and utilities 

def your_eff_know_skill(reward, cost, diff,skill, A, B):  #model for planning. Takes in diff, skill, and cost -returns effort
    sampled_effort= list()
    sampled_skill=list()
    for i in xrange(N_samples):
        d = np.random.choice(diff)
        effort,_,_=get_effort(reward, cost, d, skill, A, B)
        sampled_effort.append(effort)
        sampled_skill.append(skill)
    
    return sampled_effort

def create_priors(N_samples, tao_skill,u_skill):
    sampled_skill_high = list()
    sampled_diff_high = list()
    sampled_skill_low = list()
    sampled_diff_low = list()
    sampled_skill_low_fail=list()
    sampled_diff_low_fail=list()
    sampled_skill_high_fail=list()
    sampled_diff_high_fail=list()
    diff_samp=list()
    skill_samp=list()
    effort_samp=list()
    succes_samp=list()
    for _ in xrange(N_samples):
        skill = np.random.beta(tao_skill*u_skill,(1-u_skill)*tao_skill) #skill is a beta function
        diff = np.random.beta(.5,.5) #difficulty is a beta function
        effort,_,_ = get_effort(REWARD, COST, diff, skill, A, B) 
        high_effort = effort > threshold
        success = prob_reward(effort,diff,skill, A, B) > np.random.random()

        diff_samp.append(diff)
        skill_samp.append(skill)
        effort_samp.append(effort)
        succes_samp.append(success)
        if success and high_effort: 
            sampled_skill_high.append(skill)
            sampled_diff_high.append(diff)
        elif success and not high_effort: 
            sampled_skill_low.append(skill)
            sampled_diff_low.append(diff)
        elif not success and not high_effort: 
            sampled_skill_low_fail.append(skill)
            sampled_diff_low_fail.append(diff)
        elif not success and high_effort: 
            sampled_skill_high_fail.append(skill)
            sampled_diff_high_fail.append(diff)

    return sampled_diff_high, sampled_diff_low, sampled_diff_low_fail,sampled_diff_high_fail

### Create function that takes in free parameters and spits out model predictions

In [60]:
def run_model(cost, A, B, u_skill,tao_skill, skill):
    efforthigh_success=list()
    effortlow_success=list()
    efforthigh_fail = list()
    effortlow_fail=list()
    effortbaseline =list()
    REWARD=1
    sampled_diff_high, sampled_diff_low, sampled_diff_low_fail,sampled_diff_high_fail= create_priors(1000, tao_skill,u_skill)
    effort_high_success = your_eff_know_skill(REWARD, COST, sampled_diff_high,skill, A,B) 
    efforthigh_success.append(effort_high_success)
    effort_low_success = your_eff_know_skill(REWARD, COST, sampled_diff_low,skill,A,B) 
    effortlow_success.append(effort_low_success)
    effort_high_fail = your_eff_know_skill(REWARD, COST, sampled_diff_high_fail,skill,A,B) 
    efforthigh_fail.append(effort_high_fail)
    effort_low_fail = your_eff_know_skill(REWARD, COST, sampled_diff_low_fail,skill,A,B) 
    effortlow_fail.append(effort_low_fail)
    effort_baseline = your_eff_know_skill(REWARD, COST, np.random.beta(.5, .5, size=1000),skill,A,B)
    effortbaseline.append(effort_baseline)
    
    return efforthigh_success, effortlow_success, efforthigh_fail, effortlow_fail, effortbaseline

In [78]:
efforthigh_success, effortlow_success, efforthigh_fail, effortlow_fail, effortbaseline = run_model(.5, 6, 60, .625,20, .4)

In [79]:
 effortbaseline

[0.16161616161616163,
 0.21212121212121213,
 0.010101010101010102,
 0.22222222222222224,
 0.070707070707070718,
 0.15151515151515152,
 0.050505050505050511,
 0.010101010101010102,
 0.060606060606060608,
 0.040404040404040407,
 0.010101010101010102,
 0.010101010101010102,
 0.23232323232323235,
 0.050505050505050511,
 0.15151515151515152,
 0.11111111111111112,
 0.23232323232323235,
 0.010101010101010102,
 0.23232323232323235,
 0.17171717171717174,
 0.14141414141414144,
 0.010101010101010102,
 0.17171717171717174,
 0.11111111111111112,
 0.10101010101010102,
 0.22222222222222224,
 0.23232323232323235,
 0.23232323232323235,
 0.040404040404040407,
 0.010101010101010102,
 0.010101010101010102,
 0.060606060606060608,
 0.030303030303030304,
 0.23232323232323235,
 0.22222222222222224,
 0.19191919191919193,
 0.010101010101010102,
 0.23232323232323235,
 0.010101010101010102,
 0.20202020202020204,
 0.14141414141414144,
 0.060606060606060608,
 0.11111111111111112,
 0.22222222222222224,
 0.0202020202

In [80]:
set([type(i) for i in effortbaseline])



{list, numpy.float64}

In [81]:
for idx, val in enumerate(effortbaseline):
    if isinstance(val, list):
        print(idx)

10000


In [ ]:
COST = 0.5
B=6
A=60 
u_skill = .625 #mean of beta
tao_skill = 20 #variance of beta
skill=.4
scale = 200

### Run functions to get effort predictions per condition

In [76]:
sampled_diff_high, sampled_diff_low, sampled_diff_low_fail,sampled_diff_high_fail= create_priors(100, tao_skill,u_skill)
effort_high_success = your_eff_know_skill(REWARD, COST, sampled_diff_high,skill,A,B) 
effort_low_success = your_eff_know_skill(REWARD, COST, sampled_diff_low,skill,A,B) 
effort_high_fail = your_eff_know_skill(REWARD, COST, sampled_diff_high_fail,skill,A,B) 
effort_low_fail = your_eff_know_skill(REWARD, COST, sampled_diff_low_fail,skill,A,B) 
#effort_baseline = your_eff_know_skill(REWARD, COST, np.random.beta(tao_skill*u_skill,(1-u_skill)*tao_skill, size=1000),skill) ##change? 
effort_baseline = your_eff_know_skill(REWARD, COST, np.random.beta(.5, .5, size=100),skill,A,B) ##change? 


print np.mean(effort_high_success)
print np.mean(effort_low_success)
print np.mean(effort_high_fail)
print np.mean(effort_low_fail)
print np.mean(effort_baseline)

0.418507070707
0.26246969697
0.0
0.0446545454545
0.283003030303


In [77]:
set([type(i) for i in effort_baseline])

{numpy.float64}